# Analysis of Gasoline Prices and Consumption  
### INFO 511 — Final Project  
**Project Title:** *The Impact of Gasoline Prices on U.S. Gasoline Consumption (1993–2024)*  
**Author:** Nikolas Lee-Bishop  
**Date:** October 2025  

---

### **Notebook Purpose**
This notebook conducts statistical and predictive analysis on the cleaned and merged dataset produced during the Exploratory Data Analysis (EDA) phase.  
The goal is to quantify and interpret the relationship between U.S. gasoline prices and gasoline consumption, and to determine whether higher prices lead to lower consumption when accounting for inflation and other factors.

---

### **Notebook Structure**
1. **Data Loading & Preparation**  
   - Import the cleaned `merged_gasoline_data.csv` from the EDA notebook.  
   - Perform any final preprocessing or transformations as needed.  

2. **Descriptive Analysis**  
   - Review key descriptive statistics and visual trends.  
   - Check for outliers, residual skewness, and normality after transformations.  

3. **Correlation & Relationship Analysis**  
   - Compute correlation coefficients between nominal price, real (inflation-adjusted) price, and consumption.  
   - Visualize correlations with scatter and heatmap plots.  

4. **Regression Modeling**  
   - Build and evaluate linear regression models:  
     - **Model 1:** Nominal Price → Consumption  
     - **Model 2:** Inflation-Adjusted Price → Consumption  
   - Assess statistical significance, R² values, and residuals.  

5. **Additional Analyses (Optional)**  
   - Perform rolling-window regression or trend decomposition.  
   - Test for seasonality or lagged effects using autocorrelation or time-series regression.  

6. **Conclusion & Interpretation**  
   - Summarize findings and discuss implications of price elasticity and consumption behavior.  

---

### **Data Source**
Dataset used in this notebook was produced from the EDA process and saved as:  
`data/merged_gasoline_data.csv`

---

### **Dependencies**
```python
# Core libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import pearsonr, spearmanr

Things to do:
- update header to keep accurate
- regression
- ??

In [ ]:
import pandas as pd

merged_df = pd.read_csv("data/merged_gasoline_data.csv")

In [ ]:
''' Create a scatter plot with a LOWESS (locally weighted) smoothing curve '''

import plotly.express as px

fig = px.scatter(
    merged_df,
    x="Dollars_Per_Gallon",
    y="Gasoline_Consumption(Millions of Gallons)",
    color="Year",
    color_continuous_scale="plasma",
    opacity=0.7,
    title="Gasoline Price vs. Consumption with LOWESS Smoothing (1993–2024)",
    labels={
        "Dollars_Per_Gallon": "Gasoline Price (USD per Gallon)",
        "Gasoline_Consumption(Millions of Gallons)": "Gasoline Consumption (Millions of Gallons)"
    },
)

# Add a LOWESS smoothing line
import statsmodels.api as sm
lowess = sm.nonparametric.lowess

smoothed = lowess(
    merged_df["Gasoline_Consumption(Millions of Gallons)"],
    merged_df["Dollars_Per_Gallon"],
    frac=0.2  # adjust smoothness: smaller = more local, larger = smoother
)

fig.add_scatter(
    x=smoothed[:, 0],
    y=smoothed[:, 1],
    mode="lines",
    line=dict(color="black", width=3),
    name="LOWESS Fit"
)

# Layout customization
fig.update_layout(
    template="plotly_white",
    title_x=0.5,
    hovermode="x unified",
    xaxis=dict(showgrid=True, gridcolor="lightgray"),
    yaxis=dict(showgrid=True, gridcolor="lightgray"),
    height=600
)

fig.show()

In [ ]:
''' Price vs Consumption Scatter Plot '''

# Base scatter plot with regression line
fig = px.scatter(
    merged_df,
    x="Dollars_Per_Gallon",
    y="Gasoline_Consumption(Millions of Gallons)",
    trendline="ols",
    title="Relationship Between Gasoline Price and Consumption (1993–2024)",
    labels={
        "Dollars_Per_Gallon": "Gasoline Price (USD per Gallon)",
        "Gasoline_Consumption(Millions of Gallons)": "Gasoline Consumption (Millions of Gallons)"
    },
    opacity=0.7,
    color="Year"
)

# Extract OLS regression results from the trendline
results = px.get_trendline_results(fig)
model = results.iloc[0]["px_fit_results"]

# Extract slope, intercept, and R²
slope = model.params[1]
intercept = model.params[0]
r_squared = model.rsquared

# Add regression annotation to the plot
fig.add_annotation(
    x=merged_df["Dollars_Per_Gallon"].max() * 0.7,
    y=merged_df["Gasoline_Consumption(Millions of Gallons)"].max() * 0.95,
    text=f"y = {intercept:.2f} + {slope:.2f}x<br>R² = {r_squared:.3f}",
    showarrow=False,
    font=dict(size=12, color="black"),
    bgcolor="rgba(255,255,255,0.7)",
    bordercolor="lightgray",
    borderwidth=1
)

# Customize layout
fig.update_layout(
    template="plotly_white",
    title_x=0.5,
    hovermode="x unified",
    xaxis=dict(showgrid=True, gridcolor="lightgray"),
    yaxis=dict(showgrid=True, gridcolor="lightgray"),
    height=600
)

fig.show()